Generate assessment report

# Assessment Report

## Configure OpenAI Settings

In [1]:
import os
import openai
from dotenv import load_dotenv
# Set up Azure OpenAI
load_dotenv()

openai.api_type = "azure"

AZURE_OPENAI_API_VERSION = os.getenv("AAG_AZURE_OPENAI_API_VERSION")
openai.api_version = AZURE_OPENAI_API_VERSION

AZURE_OPENAI_API_KEY = os.getenv("AAG_AZURE_OPENAI_API_KEY").strip()
assert AZURE_OPENAI_API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = AZURE_OPENAI_API_KEY

AZURE_OPENAI_ENDPOINT = os.getenv("AAG_AZURE_OPENAI_ENDPOINT", "").strip()
assert AZURE_OPENAI_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
openai.api_base = AZURE_OPENAI_ENDPOINT

# Deployment for Chat
# DEPLOYMENT_NAME_CHAT = os.getenv('DEPLOYMENT_NAME_CHAT')
DEPLOYMENT_NAME_CHAT = os.getenv('AAG_DEPLOYMENT_NAME_CHAT_16K')

# Deployment for embedding
DEPLOYMENT_NAME_EMBEDDING = os.getenv("AAG_DEPLOYMENT_NAME_EMBEDDING")
model: str = DEPLOYMENT_NAME_EMBEDDING

# Azure AI Search (Cognitive vector store)
vector_store_address: str = os.getenv("AAG_AZURE_SEARCH_SERVICE_ENDPOINT")
vector_store_password: str = os.getenv("AAG_AZURE_SEARCH_ADMIN_KEY")
# index_name: str = "langchain-vector-arxiv-physics"

# Deployment for embedding
BING_SUBSCRIPTION_KEY = os.getenv("BING_SUBSCRIPTION_KEY")

## Azure OpenAI ChatCompletion

### Read finds from previous processed data saved in qna.json

In [6]:
import json
import os

# Load findings on a company
# file_path = './reports/qna_msft.json'
file_path = './reports/qna_zbra_003.json'

# Check if the file exists
if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        # Process the data as needed
        print(data)  # Example: printing the loaded JSON data
else:
    print(f"The file '{file_path}' does not exist.")

# data

[{'Category': 'Financial Profile:', 'Inquiries': [{'Question': 'What is the trend in revenue growth over the past few years?', 'Answer': 'Zebra Technologies has experienced revenue growth over the past few years, with a significant increase in 2021 compared to 2020.'}, {'Question': "How are the company's gross, operating, and net profit margins trending?", 'Answer': "The company's gross, operating, and net profit margins have been trending as follows:\n- Gross profit margin: $2,624 million in 2022, $2,628 million in 2021, and $2,003 million in 2020.\n- Operating profit margin: $529 million in 2022, $979 million in 2021, and $651 million in 2020.\n- Net profit margin: $463 million in 2022, $837 million in 2021, and $504 million in 2020."}, {'Question': "What is the company's operating cash flow and free cash flow?", 'Answer': 'The operating cash flow for Zebra Technologies for the years ended December 31, 2022, 2021, and 2020 were $488 million, $1,069 million, and $962 million, respecti

### Run ChatCompletion with user prompt to generate assessment reports

In [7]:
company_name = 'Zebra Technologies (Zebra)'

prompt_text = f"""
Please generate an assessment report on {company_name} based on the following questions and answers in JSON:
{data}
and the assessment report should contain the following sections:
1. Executive Summary
2. Overview of Key Findings
3. Summary of Risks Identified
4. Overall ratings on the risk level you would give, between 0 and 9 (0 - no risk, 9 - highest risk)
"""
# prompt_text = f"""
# Are you busy now?
# """
print(prompt_text)

response = openai.ChatCompletion.create(
    engine=DEPLOYMENT_NAME_CHAT,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt_text},
    ],
    max_tokens=15000,
)
report_content = response["choices"][0]["message"]["content"]

print(response["choices"][0]["message"]["content"])


Please generate an assessment report on Zebra Technologies (Zebra) based on the following questions and answers in JSON:
[{'Category': 'Financial Profile:', 'Inquiries': [{'Question': 'What is the trend in revenue growth over the past few years?', 'Answer': 'Zebra Technologies has experienced revenue growth over the past few years, with a significant increase in 2021 compared to 2020.'}, {'Question': "How are the company's gross, operating, and net profit margins trending?", 'Answer': "The company's gross, operating, and net profit margins have been trending as follows:\n- Gross profit margin: $2,624 million in 2022, $2,628 million in 2021, and $2,003 million in 2020.\n- Operating profit margin: $529 million in 2022, $979 million in 2021, and $651 million in 2020.\n- Net profit margin: $463 million in 2022, $837 million in 2021, and $504 million in 2020."}, {'Question': "What is the company's operating cash flow and free cash flow?", 'Answer': 'The operating cash flow for Zebra Techno

In [8]:
report_content

"Assessment Report: Zebra Technologies (Zebra)\n\n1. Executive Summary:\nZebra Technologies is a company that has experienced revenue growth over the past few years, with a significant increase in 2021 compared to 2020. The company's gross, operating, and net profit margins have been trending consistently. Zebra Technologies has a debt structure that includes long-term debt and current debt. The major risks disclosed by the company include interest rate risk and foreign exchange risk. The company performs well compared to its competitors in the industry. Zebra Technologies does not currently allocate profits to dividends but has a share repurchase program. The company's cash conversion cycle has been decreasing over the past five years. No pending litigations, investigations, or regulatory issues have been disclosed. The company has not highlighted specific metrics or KPIs as measures of success. Market trends or industry-specific challenges affecting the company could not be found, bu

In [13]:
word_file = file_path + '_assessment.txt'

with open(word_file, 'w') as file:
    file.write(report_content)

In [ ]:
# ! pip install python-docx

In [9]:
import json
import os
from docx import Document

new_file_path = file_path.replace('.json', '_assessment.docx')

# Create a new Word document
doc = Document()
doc.add_heading(f'Assessment Report on {company_name}', level=1)

# Add content to the Word document
doc.add_paragraph(report_content)

# Save the Word document
doc.save(new_file_path)

print(f"Assessment report saved to: {new_file_path}")


Assessment report saved to: ./reports/qna_zbra_003_assessment.docx
